# Differentiable Uncalibrated Imaging

In [ ]:
import itertools
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchkbnufft as tkbn
from skimage.data import shepp_logan_phantom
from skimage.transform import resize
from tqdm import tqdm
# import torchviz
from sigpy.mri.sim import birdcage_maps
from scipy.ndimage import label

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
resolution = (64, 64)
num_coils = 4

In [ ]:
def init_phantom(resolution):
    image = shepp_logan_phantom().astype(np.float32)  # Shape format (H,W)
    image = resize(image, resolution, anti_aliasing=True)
    image = torch.tensor(image, dtype=torch.float)
    image = (image + 1j * torch.zeros_like(image)).unsqueeze(0).unsqueeze(0)
    return image

def init_kcoords(resolution, accel=2):
    kcoords = torch.rand(int(resolution[0] * resolution[1] / accel), 2) * 2 * np.pi - np.pi    
    return kcoords

def cpx2dim(x): return torch.cat([x.real, x.imag], dim=1)
def dim2cpx(x): return x[:,:num_coils] + 1j*x[:,num_coils:]

def conjdot(a, b): return (a.conj() * b).sum().abs()

def set_background_sensitivity_to_zero(csm, image_gt):
    image_copy = image_gt.clone().cpu().numpy()
    image_copy[:, :, resolution[0]//4:resolution[0]*3//4, resolution[1]//4:resolution[1]*3//4] = 1
    labelmap, _ = label(image_copy > 0)
    fg = labelmap != 0
    fg = torch.tensor(fg, device=device)
    csm = fg * csm
    return csm, fg


In [ ]:
class FourierFeaturesEncoder(nn.Module):
  """
  Fourier features encoder.
  Module adapted from: https://github.com/swing-research/differentiable_uncalibrated_imaging/blob/main/models/implicit_neural.py
  """
  def __init__(self, ffenc_size_multiplier=10):
    super().__init__()
    self.coeffs = (torch.arange(1, ffenc_size_multiplier + 1e-12) * math.pi * 0.5).to(device)

  def forward(self, x):
    x = torch.kron(self.coeffs, x)
    return torch.hstack((torch.sin(x), torch.cos(x)))


class MeasurementRepr(nn.Module):
  """
  Measurement representation model.
  Module adapted from: https://github.com/swing-research/differentiable_uncalibrated_imaging/blob/main/models/implicit_neural.py
  """
  def __init__(self, in_features=2, hidden_features=256, hidden_blocks=6, out_features=8, ffenc_size_multiplier=10):
    
    super().__init__()

    self.ff_encoder = FourierFeaturesEncoder(ffenc_size_multiplier)
    ffm_expansion_size = 2 * in_features * ffenc_size_multiplier

    self.blocks = nn.ModuleList()
    self.blocks.append(nn.Sequential(
       nn.Linear(ffm_expansion_size, hidden_features), nn.ReLU(),
       nn.Linear(hidden_features, hidden_features),  nn.ReLU()))
    for _ in range(hidden_blocks-1):
      self.blocks.append(nn.Sequential(
         nn.Linear(hidden_features + ffm_expansion_size, hidden_features), nn.ReLU(),
         nn.Linear(hidden_features, hidden_features), nn.ReLU()))

    self.out_block = nn.Sequential(
       nn.Linear(hidden_features + ffm_expansion_size, hidden_features), nn.ReLU(),
       nn.Linear(hidden_features, int(hidden_features / 2)), nn.ReLU(),
       nn.Linear(int(hidden_features / 2), out_features))      

  def forward(self, coords):  # Shape: (ksamples, 2)
    fourier_features = self.ff_encoder(coords)
    x = fourier_features
    for block in self.blocks:
      x = block(x)
      x = torch.cat((x, fourier_features), dim=1)
    out = self.out_block(x)
    out = dim2cpx(out)  # Shape: (ksamples, coils); Dtype = complex64
    return out


class ForwardOperator:

    """ SENSE operator """

    def __init__(self, csm=None, device='cpu'):
        self.nufft = tkbn.KbNufft(im_size=resolution, device=device)
        self.inufft = tkbn.KbNufftAdjoint(im_size=resolution, device=device)        
        self.csm = csm

    def set_coords(self, kcoords):
        self.kcoords = kcoords.T
        self.interp_matrices = tkbn.calc_tensor_spmatrix(self.kcoords.clone().detach(), im_size=resolution)
        self.interp_matrices = (self.interp_matrices[0].to(kcoords.device), self.interp_matrices[1].to(kcoords.device))

    def __call__(self, image): 
        kdata = self.nufft(image, self.kcoords, smaps=self.csm)  # Shape: (1, coils, ksamples); Dtype = complex64
        kdata = kdata.squeeze(0).T  # Shape: (ksamples, coils); Dtype = complex64
        return kdata
    
    def H(self, kdata):              # Shape: (ksamples, 1); Dtype = complex64
        kdata = kdata.T.unsqueeze(0) # Shape: (1, 1, ksamples); Dtype = complex64
        image = self.inufft(kdata, self.kcoords, self.interp_matrices, smaps=self.csm)  # Shape: (1, 1, H, W); Dtype = complex64
        return image


class ReconstructorSimple:
    def __call__(self, kdata, A): return A.H(kdata)


class ReconstructorCGSENSE:
    """ CG-SENSE reconstruction """
    def __call__(self, kdata, A):
      a = A.H(kdata)
      b = torch.zeros_like(a)
      p = a.clone()
      r = a.clone()
      rdotr_prev = conjdot(r, r)
      for _ in range(10):          
        q = A.H(A(p))
        b = b + (rdotr_prev / conjdot(p, q)) * p
        r = r - (rdotr_prev / conjdot(p, q)) * q
        rdotr = conjdot(r, r)
        p = r + (rdotr / rdotr_prev) * p          
        rdotr_prev = rdotr
      return b

In [ ]:
# Init stuff
image_gt = init_phantom(resolution).to(device)         # Shape: (1, 1, H, W); Dtype = complex64
csm = torch.tensor(birdcage_maps((num_coils, resolution[0], resolution[1]), r=1.1, dtype=np.complex64), device=device)  # Shape format (C,H,W)
csm, fg = set_background_sensitivity_to_zero(csm, image_gt)

R = MeasurementRepr().to(device)
A = ForwardOperator(csm, device)
# G = ReconstructorSimple()
G = ReconstructorCGSENSE()

kcoords_true = init_kcoords(resolution, accel=1.5).to(device)   # Shape: (ksamples, 2)
A.set_coords(kcoords_true)
kdata_true = A(image_gt).clone().detach()  # Shape: (ksamples, 1); Dtype = complex64

# Simulate miscalibration
kcoords_miscalib = kcoords_true + torch.randn_like(kcoords_true) * torch.pi * 0.01
kcoords_miscalib = torch.clip(kcoords_miscalib, -torch.pi, torch.pi)
kcoords = kcoords_miscalib.clone().detach().to(device)
kcoords.requires_grad = True

# Sanity check
A.set_coords(kcoords_miscalib); recon_uncorrected = G(kdata_true, A)
A.set_coords(kcoords_true);     recon_true = G(kdata_true, A)
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].imshow(recon_uncorrected.abs().squeeze().detach().cpu()); axs[0].set_title("Recon: uncorrected")
axs[1].imshow(recon_true.abs().squeeze().detach().cpu()); axs[1].set_title("Recon: truth")
plt.show()

### Test framework

In [ ]:
def loss_fitting(kdata, kdata_pred_r):             return torch.norm(kdata - kdata_pred_r, p=2)
def loss_consistency(kdata_pred_r, kdata_pred_ag): return torch.norm(kdata_pred_r - kdata_pred_ag, p=2)

In [ ]:
# Optimization prep
params = itertools.chain([kcoords], R.parameters())
optimizer = torch.optim.Adam(params, lr=1e-4)
consistency_weight = 1e-1

# Optimization loop
num_iters = 5
loss_history = []
kcoord_errors = []
for it in tqdm(range(num_iters)):
        
    optimizer.zero_grad()

    A.set_coords(kcoords)
    kdata_pred_r = R(kcoords)  # Shape: {ksamples, 1}
    kdata_pred_ag = A(G(kdata_pred_r, A))  # Shape: {ksamples, 1}
    loss = loss_fitting(kdata_true, kdata_pred_r) + consistency_weight * loss_consistency(kdata_pred_r, kdata_pred_ag)
    
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    kcoord_errors.append(F.mse_loss(kcoords.clone().detach(), kcoords_true).item())


# Results
print(loss_history[-1])
print(kcoord_errors[-1])
# plt.plot(kcoord_errors); plt.show()

A.set_coords(kcoords_miscalib); recon_uncorrected_inr = G(R(kcoords_miscalib), A)
A.set_coords(kcoords);          recon_corrected_inr = G(R(kcoords), A)
A.set_coords(kcoords_true);     recon_true = G(kdata_true, A)

fig, axs = plt.subplots(1, 5, figsize=(24, 4))
axs[0].plot(loss_history); axs[0].set_title("Loss")
axs[1].imshow(recon_uncorrected.abs().squeeze().detach().cpu()); axs[1].set_title("Recon: uncorrected")
axs[2].imshow(recon_uncorrected_inr.abs().squeeze().detach().cpu()); axs[1].set_title("Recon: uncorrected + INR")
axs[3].imshow(recon_corrected_inr.abs().squeeze().detach().cpu()); axs[2].set_title("Recon: corrected + INR")
axs[4].imshow(recon_true.abs().squeeze().detach().cpu()); axs[3].set_title("Recon: truth")
plt.show()